In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import glob
import os

In [2]:
df = pd.read_csv('private.txt', header=None, error_bad_lines=False, skip_blank_lines=False)
df

b'Skipping line 3: expected 1 fields, saw 9\n'


,0
0,"<div class=""competition""><div class=""competiti..."
1,"<div class=""competition""><div class=""competiti..."
2,"<div class=""competition""><div class=""competiti..."
3,https://www.kaggle.com/c/LANL-Earthquake-Predi...


In [3]:
dfinfo = list(df[0])
len(dfinfo)

4

In [4]:
#En este codigo se parsea ese HTML y se extrae exactamente lo que se quiere. 
# Es como utilizar BeautifulSoup in python.
# Probar y ver mas información.

mydict = {}

lista = []

for x in range(len(dfinfo)):
    htmltxt = dfinfo[x]
    soup = BeautifulSoup(htmltxt, "html.parser")
    rank = soup.find_all("td",{"competition-leaderboard__td-rank"})
    
    mydict_aux = []
    for i in range(len(rank)):
    #for i in range(10):
        #htmltxt = dfinfo[i]
        #soup = BeautifulSoup(test_1, "html.parser")
        mydict = {}
        # 1. Competition Title 
        mydict["title"] = soup.find("h1").text

        # 2. Rank
        rank = soup.find_all("td",{"competition-leaderboard__td-rank"})[i]
        mydict["rank_leaderboard"] =rank.text
        
        # 3. Name_team
        name_team = soup.find_all("td",{"data-th":"Team Name"})[i]
        mydict["name_team"] = name_team.get_text()
        
        
        
        # 4. Team_members

        user = {}

        users = []
        for h in soup.find_all("td",{"competition-leaderboard__td-members"})[i]:
            user = {}
            a = h.find('a')

            b = a.find_all("img")
            new_list = []
            for j in b:
                new_list.append(j.attrs['alt'])
            user["name"] = new_list[0]
            user["tier"] = new_list[1]

            users.append(user)

        mydict["team_members"] = users
        
        
        ## 5. Score
        score = soup.find_all("td",{"competition-leaderboard__td-score"})[i]
        mydict["score_leaderboard"] = score.get_text()

        ## 5.2 Evaluation_Score
        Evaluation_Score = soup.find_all("th",{"competition-leaderboard__th-score"})[0]
        mydict["Evaluation_Score"] = Evaluation_Score.find("span").attrs["data-tooltip"]

        ## 6. Entries
        entries = soup.find_all("td",{"competition-leaderboard__td-entries"})[i]
        mydict["entries"] = entries.get_text()
        
        ## 7.Last Entry 
        Last_Entry = soup.find_all("td",{"competition-leaderboard__td-last"})[0]
        mydict["last_entry"] = Last_Entry.find("span").attrs['title']
        
        
        ## 8.Position_Change
        position_change = soup.find_all("td",{"competition-leaderboard__td-change"})[i]
        a = position_change.find("span")
        

        while True:
            try:
                mydict["position_change"] = a.text
                break
            except AttributeError :
                mydict["position_change"] = "-"
                break
        
        while True:
            try:
                aux_a = position_change.find_all("span")
                #print(aux_a[1].attrs["class"][0])
                if aux_a[1].attrs["class"][0] == "position-change__risen":
                    mydict["position_change_reference"] = "+"
                elif aux_a[1].attrs["class"][0] == "position-change__fallen": 
                    mydict["position_change_reference"] = "-"
                else: 
                    mydict["position_change_reference"] = "="
                break
            except IndexError :
                mydict["position_change_reference"] = "="
                break
                
        
        mydict_aux.append(mydict)

    
    lista.append(mydict_aux)

C:\Users\Usuario\Anaconda3\lib\site-packages\bs4\__init__.py:282: UserWarning: "https://www.kaggle.com/c/LANL-Earthquake-Prediction/leaderboard" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


In [5]:
frames = []
pd.set_option('display.max_colwidth', -1)
for d in range(len(lista)):

    dt = pd.DataFrame(lista[d])
    frames.append(dt)
    
dt = pd.concat(frames)   
dt

,Evaluation_Score,entries,last_entry,name_team,position_change,position_change_reference,rank_leaderboard,score_leaderboard,team_members,title
0,,130,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de verano de Europa central),Megvii Research Nanjing,—,=,1,0.10342,"[{'name': 'Megvii Research Nanjing', 'tier': 'novice tier'}, {'name': 'Megvii Research Nanjing', 'tier': 'novice tier'}, {'name': 'Megvii Research Nanjing', 'tier': 'novice tier'}, {'name': 'Megvii Research Nanjing', 'tier': 'novice tier'}, {'name': 'Megvii Research Nanjing', 'tier': 'novice tier'}]",iNaturalist 2019 at FGVC6
1,,24,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de verano de Europa central),LearnFromScratch,—,=,2,0.11298,"[{'name': 'LearnFromScratch', 'tier': 'novice tier'}, {'name': 'Scratch', 'tier': 'novice tier'}]",iNaturalist 2019 at FGVC6
2,,28,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de verano de Europa central),Number Six,—,=,3,0.12696,"[{'name': 'Number Six', 'tier': 'novice tier'}]",iNaturalist 2019 at FGVC6
3,,16,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de verano de Europa central),ilcve21,—,=,4,0.13121,"[{'name': 'wkcn', 'tier': 'novice tier'}, {'name': 'ilcve21', 'tier': 'novice tier'}, {'name': 'isee', 'tier': 'novice tier'}]",iNaturalist 2019 at FGVC6
4,,5,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de verano de Europa central),Hugo Touvron,—,=,5,0.13423,"[{'name': 'Hugo Touvron', 'tier': 'contributor tier'}]",iNaturalist 2019 at FGVC6
5,,25,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de verano de Europa central),Olia's,1,+,6,0.13870,"[{'name': 'Elayne Yang', 'tier': 'novice tier'}, {'name': 'Wenjing Ke', 'tier': 'novice tier'}, {'name': 'Ranran Huang', 'tier': 'novice tier'}]",iNaturalist 2019 at FGVC6
6,,47,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de verano de Europa central),Gabrichy,1,-,7,0.13959,"[{'name': 'Gabrichy', 'tier': 'novice tier'}, {'name': 'qzhb', 'tier': 'novice tier'}, {'name': 'ezhuo', 'tier': 'novice tier'}]",iNaturalist 2019 at FGVC6
7,,70,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de verano de Europa central),mochi,—,=,8,0.15234,"[{'name': 'sxr2017', 'tier': 'novice tier'}, {'name': 'boxer', 'tier': 'novice tier'}]",iNaturalist 2019 at FGVC6
8,,8,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de verano de Europa central),Sphinx,—,=,9,0.15866,"[{'name': 'Sphinx', 'tier': 'novice tier'}]",iNaturalist 2019 at FGVC6
9,,33,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de verano de Europa central),CVJena,2,+,10,0.15955,"[{'name': 'Callidior', 'tier': 'expert tier'}, {'name': 'DiKorsch', 'tier': 'novice tier'}]",iNaturalist 2019 at FGVC6


In [3]:
dt.to_csv("Dt_Teams_Private_Info_Raw.csv")

NameError: name 'df1' is not defined

## COMIENZO DE LA LIMPIEZA

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import glob
import os

In [2]:
dt=pd.read_csv('Dt_Teams_Private_Info_Raw.csv')
dt

,Unnamed: 0,Evaluation_Score,entries,last_entry,name_team,position_change,position_change_reference,rank_leaderboard,score_leaderboard,team_members,title
0,0,NaN,130.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,Megvii Research Nanjing,—,=,1.0,0.10342,"[{'name': 'Megvii Research Nanjing', 'tier': '...",iNaturalist 2019 at FGVC6
1,1,NaN,24.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,LearnFromScratch,—,=,2.0,0.11298,"[{'name': 'LearnFromScratch', 'tier': 'novice ...",iNaturalist 2019 at FGVC6
2,2,NaN,28.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,Number Six,—,=,3.0,0.12696,"[{'name': 'Number Six', 'tier': 'novice tier'}]",iNaturalist 2019 at FGVC6
3,3,NaN,16.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,ilcve21,—,=,4.0,0.13121,"[{'name': 'wkcn', 'tier': 'novice tier'}, {'na...",iNaturalist 2019 at FGVC6
4,4,NaN,5.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,Hugo Touvron,—,=,5.0,0.13423,"[{'name': 'Hugo Touvron', 'tier': 'contributor...",iNaturalist 2019 at FGVC6
5,5,NaN,25.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,Olia's,1,+,6.0,0.13870,"[{'name': 'Elayne Yang', 'tier': 'novice tier'...",iNaturalist 2019 at FGVC6
6,6,NaN,47.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,Gabrichy,1,-,7.0,0.13959,"[{'name': 'Gabrichy', 'tier': 'novice tier'}, ...",iNaturalist 2019 at FGVC6
7,7,NaN,70.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,mochi,—,=,8.0,0.15234,"[{'name': 'sxr2017', 'tier': 'novice tier'}, {...",iNaturalist 2019 at FGVC6
8,8,NaN,8.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,Sphinx,—,=,9.0,0.15866,"[{'name': 'Sphinx', 'tier': 'novice tier'}]",iNaturalist 2019 at FGVC6
9,9,NaN,33.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,CVJena,2,+,10.0,0.15955,"[{'name': 'Callidior', 'tier': 'expert tier'},...",iNaturalist 2019 at FGVC6


In [3]:
dt.shape

(795, 11)

In [4]:
print(dt.isnull().sum())

Unnamed: 0                     0
Evaluation_Score             216
entries                        3
last_entry                     0
name_team                      0
position_change                0
position_change_reference      0
rank_leaderboard               3
score_leaderboard              0
team_members                   0
title                          0
dtype: int64


In [5]:
dt.dtypes

Unnamed: 0                     int64
Evaluation_Score              object
entries                      float64
last_entry                    object
name_team                     object
position_change               object
position_change_reference     object
rank_leaderboard             float64
score_leaderboard            float64
team_members                  object
title                         object
dtype: object

In [6]:
dt.columns

Index(['Unnamed: 0', 'Evaluation_Score', 'entries', 'last_entry', 'name_team',
       'position_change', 'position_change_reference', 'rank_leaderboard',
       'score_leaderboard', 'team_members', 'title'],
      dtype='object')

In [7]:
dt.head()

,Unnamed: 0,Evaluation_Score,entries,last_entry,name_team,position_change,position_change_reference,rank_leaderboard,score_leaderboard,team_members,title
0,0,NaN,130.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,Megvii Research Nanjing,—,=,1.0,0.10342,"[{'name': 'Megvii Research Nanjing', 'tier': '...",iNaturalist 2019 at FGVC6
1,1,NaN,24.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,LearnFromScratch,—,=,2.0,0.11298,"[{'name': 'LearnFromScratch', 'tier': 'novice ...",iNaturalist 2019 at FGVC6
2,2,NaN,28.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,Number Six,—,=,3.0,0.12696,"[{'name': 'Number Six', 'tier': 'novice tier'}]",iNaturalist 2019 at FGVC6
3,3,NaN,16.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,ilcve21,—,=,4.0,0.13121,"[{'name': 'wkcn', 'tier': 'novice tier'}, {'na...",iNaturalist 2019 at FGVC6
4,4,NaN,5.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,Hugo Touvron,—,=,5.0,0.13423,"[{'name': 'Hugo Touvron', 'tier': 'contributor...",iNaturalist 2019 at FGVC6


In [8]:
#Limpiamos 'position_change', dejando todos con valores numericos y eliminando el espacio en blanco
# para posteriormente cambiar el tipo a 'int'
dt.position_change.unique()

array(['—', ' 1', ' 2', '-', ' 3', ' 5', ' 14', ' 7', ' 9', ' 4', ' 11',
       ' 10', ' 23', ' 40', ' 19', ' 12', ' 6', ' 126', ' 13', ' 8',
       ' 25', ' 24', ' 26', ' 29', ' 15', ' 99', ' 105', ' 64', ' 116',
       ' 34', ' 98', ' 106', ' 52', ' 56', ' 50', ' 155', ' 70', ' 164',
       ' 73', ' 79', ' 27', ' 54', ' 93', ' 21', ' 124', ' 16', ' 117',
       ' 30', ' 131', ' 45', ' 107', ' 179', ' 163', ' 154', ' 32', ' 49',
       ' 33', ' 76', ' 22', ' 48', ' 68', ' 88', ' 87', ' 31', ' 51',
       ' 125', ' 69', ' 17', ' 35', ' 177', ' 83', ' 81', ' 42', ' 74',
       ' 103', ' 102', ' 101', ' 96', ' 94', ' 120', ' 36', ' 97', ' 62',
       ' 138', ' 77', ' 37', ' 84', ' 20', ' 108', ' 143', ' 41', ' 134',
       ' 66', ' 55', ' 43', ' 65', ' 75', ' 158', ' 58', ' 71', ' 86',
       ' 59', ' 174', ' 127', ' 38', ' 111', ' 115', ' 46', ' 92', ' 18',
       ' 60', ' 39'], dtype=object)

In [9]:
dt["position_change"] = dt["position_change"].str.replace('-', '0').str.replace('—','0')
dt.head()

,Unnamed: 0,Evaluation_Score,entries,last_entry,name_team,position_change,position_change_reference,rank_leaderboard,score_leaderboard,team_members,title
0,0,NaN,130.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,Megvii Research Nanjing,0,=,1.0,0.10342,"[{'name': 'Megvii Research Nanjing', 'tier': '...",iNaturalist 2019 at FGVC6
1,1,NaN,24.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,LearnFromScratch,0,=,2.0,0.11298,"[{'name': 'LearnFromScratch', 'tier': 'novice ...",iNaturalist 2019 at FGVC6
2,2,NaN,28.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,Number Six,0,=,3.0,0.12696,"[{'name': 'Number Six', 'tier': 'novice tier'}]",iNaturalist 2019 at FGVC6
3,3,NaN,16.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,ilcve21,0,=,4.0,0.13121,"[{'name': 'wkcn', 'tier': 'novice tier'}, {'na...",iNaturalist 2019 at FGVC6
4,4,NaN,5.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,Hugo Touvron,0,=,5.0,0.13423,"[{'name': 'Hugo Touvron', 'tier': 'contributor...",iNaturalist 2019 at FGVC6


In [10]:
dt.columns

Index(['Unnamed: 0', 'Evaluation_Score', 'entries', 'last_entry', 'name_team',
       'position_change', 'position_change_reference', 'rank_leaderboard',
       'score_leaderboard', 'team_members', 'title'],
      dtype='object')

In [11]:
dt["position_change"] = dt["position_change"].str.strip()

In [12]:
dt.position_change.unique()

array(['0', '1', '2', '3', '5', '14', '7', '9', '4', '11', '10', '23',
       '40', '19', '12', '6', '126', '13', '8', '25', '24', '26', '29',
       '15', '99', '105', '64', '116', '34', '98', '106', '52', '56',
       '50', '155', '70', '164', '73', '79', '27', '54', '93', '21',
       '124', '16', '117', '30', '131', '45', '107', '179', '163', '154',
       '32', '49', '33', '76', '22', '48', '68', '88', '87', '31', '51',
       '125', '69', '17', '35', '177', '83', '81', '42', '74', '103',
       '102', '101', '96', '94', '120', '36', '97', '62', '138', '77',
       '37', '84', '20', '108', '143', '41', '134', '66', '55', '43',
       '65', '75', '158', '58', '71', '86', '59', '174', '127', '38',
       '111', '115', '46', '92', '18', '60', '39'], dtype=object)

In [13]:
dt.dtypes

Unnamed: 0                     int64
Evaluation_Score              object
entries                      float64
last_entry                    object
name_team                     object
position_change               object
position_change_reference     object
rank_leaderboard             float64
score_leaderboard            float64
team_members                  object
title                         object
dtype: object

In [14]:
dt['change_position'] = dt['position_change_reference'] + dt['position_change']
dt

,Unnamed: 0,Evaluation_Score,entries,last_entry,name_team,position_change,position_change_reference,rank_leaderboard,score_leaderboard,team_members,title,change_position
0,0,NaN,130.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,Megvii Research Nanjing,0,=,1.0,0.10342,"[{'name': 'Megvii Research Nanjing', 'tier': '...",iNaturalist 2019 at FGVC6,=0
1,1,NaN,24.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,LearnFromScratch,0,=,2.0,0.11298,"[{'name': 'LearnFromScratch', 'tier': 'novice ...",iNaturalist 2019 at FGVC6,=0
2,2,NaN,28.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,Number Six,0,=,3.0,0.12696,"[{'name': 'Number Six', 'tier': 'novice tier'}]",iNaturalist 2019 at FGVC6,=0
3,3,NaN,16.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,ilcve21,0,=,4.0,0.13121,"[{'name': 'wkcn', 'tier': 'novice tier'}, {'na...",iNaturalist 2019 at FGVC6,=0
4,4,NaN,5.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,Hugo Touvron,0,=,5.0,0.13423,"[{'name': 'Hugo Touvron', 'tier': 'contributor...",iNaturalist 2019 at FGVC6,=0
5,5,NaN,25.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,Olia's,1,+,6.0,0.13870,"[{'name': 'Elayne Yang', 'tier': 'novice tier'...",iNaturalist 2019 at FGVC6,+1
6,6,NaN,47.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,Gabrichy,1,-,7.0,0.13959,"[{'name': 'Gabrichy', 'tier': 'novice tier'}, ...",iNaturalist 2019 at FGVC6,-1
7,7,NaN,70.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,mochi,0,=,8.0,0.15234,"[{'name': 'sxr2017', 'tier': 'novice tier'}, {...",iNaturalist 2019 at FGVC6,=0
8,8,NaN,8.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,Sphinx,0,=,9.0,0.15866,"[{'name': 'Sphinx', 'tier': 'novice tier'}]",iNaturalist 2019 at FGVC6,=0
9,9,NaN,33.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,CVJena,2,+,10.0,0.15955,"[{'name': 'Callidior', 'tier': 'expert tier'},...",iNaturalist 2019 at FGVC6,+2


In [15]:
dt["change_position"] = dt["change_position"].str.replace("=", "")

In [16]:
dt

,Unnamed: 0,Evaluation_Score,entries,last_entry,name_team,position_change,position_change_reference,rank_leaderboard,score_leaderboard,team_members,title,change_position
0,0,NaN,130.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,Megvii Research Nanjing,0,=,1.0,0.10342,"[{'name': 'Megvii Research Nanjing', 'tier': '...",iNaturalist 2019 at FGVC6,0
1,1,NaN,24.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,LearnFromScratch,0,=,2.0,0.11298,"[{'name': 'LearnFromScratch', 'tier': 'novice ...",iNaturalist 2019 at FGVC6,0
2,2,NaN,28.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,Number Six,0,=,3.0,0.12696,"[{'name': 'Number Six', 'tier': 'novice tier'}]",iNaturalist 2019 at FGVC6,0
3,3,NaN,16.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,ilcve21,0,=,4.0,0.13121,"[{'name': 'wkcn', 'tier': 'novice tier'}, {'na...",iNaturalist 2019 at FGVC6,0
4,4,NaN,5.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,Hugo Touvron,0,=,5.0,0.13423,"[{'name': 'Hugo Touvron', 'tier': 'contributor...",iNaturalist 2019 at FGVC6,0
5,5,NaN,25.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,Olia's,1,+,6.0,0.13870,"[{'name': 'Elayne Yang', 'tier': 'novice tier'...",iNaturalist 2019 at FGVC6,+1
6,6,NaN,47.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,Gabrichy,1,-,7.0,0.13959,"[{'name': 'Gabrichy', 'tier': 'novice tier'}, ...",iNaturalist 2019 at FGVC6,-1
7,7,NaN,70.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,mochi,0,=,8.0,0.15234,"[{'name': 'sxr2017', 'tier': 'novice tier'}, {...",iNaturalist 2019 at FGVC6,0
8,8,NaN,8.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,Sphinx,0,=,9.0,0.15866,"[{'name': 'Sphinx', 'tier': 'novice tier'}]",iNaturalist 2019 at FGVC6,0
9,9,NaN,33.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,CVJena,2,+,10.0,0.15955,"[{'name': 'Callidior', 'tier': 'expert tier'},...",iNaturalist 2019 at FGVC6,+2


In [17]:
dt["change_position"] = dt["change_position"].astype(str).astype(int)
dt

,Unnamed: 0,Evaluation_Score,entries,last_entry,name_team,position_change,position_change_reference,rank_leaderboard,score_leaderboard,team_members,title,change_position
0,0,NaN,130.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,Megvii Research Nanjing,0,=,1.0,0.10342,"[{'name': 'Megvii Research Nanjing', 'tier': '...",iNaturalist 2019 at FGVC6,0
1,1,NaN,24.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,LearnFromScratch,0,=,2.0,0.11298,"[{'name': 'LearnFromScratch', 'tier': 'novice ...",iNaturalist 2019 at FGVC6,0
2,2,NaN,28.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,Number Six,0,=,3.0,0.12696,"[{'name': 'Number Six', 'tier': 'novice tier'}]",iNaturalist 2019 at FGVC6,0
3,3,NaN,16.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,ilcve21,0,=,4.0,0.13121,"[{'name': 'wkcn', 'tier': 'novice tier'}, {'na...",iNaturalist 2019 at FGVC6,0
4,4,NaN,5.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,Hugo Touvron,0,=,5.0,0.13423,"[{'name': 'Hugo Touvron', 'tier': 'contributor...",iNaturalist 2019 at FGVC6,0
5,5,NaN,25.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,Olia's,1,+,6.0,0.13870,"[{'name': 'Elayne Yang', 'tier': 'novice tier'...",iNaturalist 2019 at FGVC6,1
6,6,NaN,47.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,Gabrichy,1,-,7.0,0.13959,"[{'name': 'Gabrichy', 'tier': 'novice tier'}, ...",iNaturalist 2019 at FGVC6,-1
7,7,NaN,70.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,mochi,0,=,8.0,0.15234,"[{'name': 'sxr2017', 'tier': 'novice tier'}, {...",iNaturalist 2019 at FGVC6,0
8,8,NaN,8.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,Sphinx,0,=,9.0,0.15866,"[{'name': 'Sphinx', 'tier': 'novice tier'}]",iNaturalist 2019 at FGVC6,0
9,9,NaN,33.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,CVJena,2,+,10.0,0.15955,"[{'name': 'Callidior', 'tier': 'expert tier'},...",iNaturalist 2019 at FGVC6,2


Una vez que creamos la columna 'change_position' con los valores que indican el cambio en la clasificiación final, borramos las columnas 'position_change' y 'position_change_reference', ya que la columna generada es una combinación de estas dos.

In [18]:
dt=dt.drop(['position_change', 'position_change_reference'], axis=1)
dt.head()

,Unnamed: 0,Evaluation_Score,entries,last_entry,name_team,rank_leaderboard,score_leaderboard,team_members,title,change_position
0,0,NaN,130.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,Megvii Research Nanjing,1.0,0.10342,"[{'name': 'Megvii Research Nanjing', 'tier': '...",iNaturalist 2019 at FGVC6,0
1,1,NaN,24.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,LearnFromScratch,2.0,0.11298,"[{'name': 'LearnFromScratch', 'tier': 'novice ...",iNaturalist 2019 at FGVC6,0
2,2,NaN,28.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,Number Six,3.0,0.12696,"[{'name': 'Number Six', 'tier': 'novice tier'}]",iNaturalist 2019 at FGVC6,0
3,3,NaN,16.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,ilcve21,4.0,0.13121,"[{'name': 'wkcn', 'tier': 'novice tier'}, {'na...",iNaturalist 2019 at FGVC6,0
4,4,NaN,5.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,Hugo Touvron,5.0,0.13423,"[{'name': 'Hugo Touvron', 'tier': 'contributor...",iNaturalist 2019 at FGVC6,0


In [19]:
dt.columns

Index(['Unnamed: 0', 'Evaluation_Score', 'entries', 'last_entry', 'name_team',
       'rank_leaderboard', 'score_leaderboard', 'team_members', 'title',
       'change_position'],
      dtype='object')

In [20]:
dt.dtypes

Unnamed: 0             int64
Evaluation_Score      object
entries              float64
last_entry            object
name_team             object
rank_leaderboard     float64
score_leaderboard    float64
team_members          object
title                 object
change_position        int32
dtype: object

In [21]:
dt.columns

Index(['Unnamed: 0', 'Evaluation_Score', 'entries', 'last_entry', 'name_team',
       'rank_leaderboard', 'score_leaderboard', 'team_members', 'title',
       'change_position'],
      dtype='object')

In [22]:
#REalizamos los cambios oportunos en la columna 'last_entry' de forma que queden en formato 'date'
dt['date'] = dt['last_entry'].str[4:15]

In [23]:
dt.head()

,Unnamed: 0,Evaluation_Score,entries,last_entry,name_team,rank_leaderboard,score_leaderboard,team_members,title,change_position,date
0,0,NaN,130.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,Megvii Research Nanjing,1.0,0.10342,"[{'name': 'Megvii Research Nanjing', 'tier': '...",iNaturalist 2019 at FGVC6,0,Jun 10 2019
1,1,NaN,24.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,LearnFromScratch,2.0,0.11298,"[{'name': 'LearnFromScratch', 'tier': 'novice ...",iNaturalist 2019 at FGVC6,0,Jun 10 2019
2,2,NaN,28.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,Number Six,3.0,0.12696,"[{'name': 'Number Six', 'tier': 'novice tier'}]",iNaturalist 2019 at FGVC6,0,Jun 10 2019
3,3,NaN,16.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,ilcve21,4.0,0.13121,"[{'name': 'wkcn', 'tier': 'novice tier'}, {'na...",iNaturalist 2019 at FGVC6,0,Jun 10 2019
4,4,NaN,5.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,Hugo Touvron,5.0,0.13423,"[{'name': 'Hugo Touvron', 'tier': 'contributor...",iNaturalist 2019 at FGVC6,0,Jun 10 2019


In [24]:
dt.columns

Index(['Unnamed: 0', 'Evaluation_Score', 'entries', 'last_entry', 'name_team',
       'rank_leaderboard', 'score_leaderboard', 'team_members', 'title',
       'change_position', 'date'],
      dtype='object')

In [25]:
dt['date'] = pd.to_datetime(dt['date'])

In [26]:
dt

,Unnamed: 0,Evaluation_Score,entries,last_entry,name_team,rank_leaderboard,score_leaderboard,team_members,title,change_position,date
0,0,NaN,130.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,Megvii Research Nanjing,1.0,0.10342,"[{'name': 'Megvii Research Nanjing', 'tier': '...",iNaturalist 2019 at FGVC6,0,2019-06-10
1,1,NaN,24.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,LearnFromScratch,2.0,0.11298,"[{'name': 'LearnFromScratch', 'tier': 'novice ...",iNaturalist 2019 at FGVC6,0,2019-06-10
2,2,NaN,28.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,Number Six,3.0,0.12696,"[{'name': 'Number Six', 'tier': 'novice tier'}]",iNaturalist 2019 at FGVC6,0,2019-06-10
3,3,NaN,16.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,ilcve21,4.0,0.13121,"[{'name': 'wkcn', 'tier': 'novice tier'}, {'na...",iNaturalist 2019 at FGVC6,0,2019-06-10
4,4,NaN,5.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,Hugo Touvron,5.0,0.13423,"[{'name': 'Hugo Touvron', 'tier': 'contributor...",iNaturalist 2019 at FGVC6,0,2019-06-10
5,5,NaN,25.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,Olia's,6.0,0.13870,"[{'name': 'Elayne Yang', 'tier': 'novice tier'...",iNaturalist 2019 at FGVC6,1,2019-06-10
6,6,NaN,47.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,Gabrichy,7.0,0.13959,"[{'name': 'Gabrichy', 'tier': 'novice tier'}, ...",iNaturalist 2019 at FGVC6,-1,2019-06-10
7,7,NaN,70.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,mochi,8.0,0.15234,"[{'name': 'sxr2017', 'tier': 'novice tier'}, {...",iNaturalist 2019 at FGVC6,0,2019-06-10
8,8,NaN,8.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,Sphinx,9.0,0.15866,"[{'name': 'Sphinx', 'tier': 'novice tier'}]",iNaturalist 2019 at FGVC6,0,2019-06-10
9,9,NaN,33.0,Mon Jun 10 2019 15:59:54 GMT+0200 (hora de ver...,CVJena,10.0,0.15955,"[{'name': 'Callidior', 'tier': 'expert tier'},...",iNaturalist 2019 at FGVC6,2,2019-06-10


In [27]:
dt.columns

Index(['Unnamed: 0', 'Evaluation_Score', 'entries', 'last_entry', 'name_team',
       'rank_leaderboard', 'score_leaderboard', 'team_members', 'title',
       'change_position', 'date'],
      dtype='object')

In [28]:
#Eliminamos columnas 'last_entry' (datos en 'date' y 'time') y 'Evaluation_Score', renombrando Unnamed: 0 que 
#contiene la información de esta.
dt=dt.drop(['last_entry', 'Unnamed: 0'], axis=1)

In [29]:
dt.head()

,Evaluation_Score,entries,name_team,rank_leaderboard,score_leaderboard,team_members,title,change_position,date
0,NaN,130.0,Megvii Research Nanjing,1.0,0.10342,"[{'name': 'Megvii Research Nanjing', 'tier': '...",iNaturalist 2019 at FGVC6,0,2019-06-10
1,NaN,24.0,LearnFromScratch,2.0,0.11298,"[{'name': 'LearnFromScratch', 'tier': 'novice ...",iNaturalist 2019 at FGVC6,0,2019-06-10
2,NaN,28.0,Number Six,3.0,0.12696,"[{'name': 'Number Six', 'tier': 'novice tier'}]",iNaturalist 2019 at FGVC6,0,2019-06-10
3,NaN,16.0,ilcve21,4.0,0.13121,"[{'name': 'wkcn', 'tier': 'novice tier'}, {'na...",iNaturalist 2019 at FGVC6,0,2019-06-10
4,NaN,5.0,Hugo Touvron,5.0,0.13423,"[{'name': 'Hugo Touvron', 'tier': 'contributor...",iNaturalist 2019 at FGVC6,0,2019-06-10


In [30]:
dt.Evaluation_Score.unique()

array([nan, 'IntersectionOverUnionObjectSegmentationWithClassification',
       'Macro F-Score'], dtype=object)

En 'Evaluation_Score' solo hay tres tipos de datos.

In [31]:
dt.to_csv('Dt_Teams_Private_Info_Raw_LIMPIO.csv')

Datos:

-Evaluation_Score: 

-entries: Número de entradas que sube cada equipo en cada competición.

-name_team: Nombre del equipo participante.

-position_change: Cambio de posición del Leaderboard public vs Leaderboard private.

-position_change_reference: Variación del equipo del public respecto al private:
          
           "=": si no ha habido cambios en su clasificación final
           "+": si ha subido puestos en la clasificación final
           "-": si ha bajado puestos en la clasificación final
         
-rank_leaderboard: Posición final en Leaderboard Private.  

-score_leaderboard: Puntuación final en Leaderboard Private.

-team_members: Miembros del equpipo participante.

-title: Título de la competición(Coincide con el título en Dataframe_Competitions_INFO, para el posterior 'join')

-date: Fecha de 'last_entry'
           

#### A continuación, agrupamos las competiciones ('title') para obtener el número total de entries de cada una de ellas.

In [32]:
dt.shape

(795, 9)

In [33]:
print(dt.isnull().sum())
#Hemos perdido 3 entries y 3 rank_leaderboard, sobre un total de 795...DE MOMENTO NO NOS PREOCUPA

Evaluation_Score     216
entries                3
name_team              0
rank_leaderboard       3
score_leaderboard      0
team_members           0
title                  0
change_position        0
date                   0
dtype: int64


In [34]:
df_entries_per_comp = dt[['title','entries']].groupby('title').sum()
df_entries_per_comp.reset_index(level=0, inplace=True)
df_entries_per_comp.head()

,title,entries
0,iMaterialist (Fashion) 2019 at FGVC6,3497.0
1,iNaturalist 2019 at FGVC6,1477.0
2,iWildCam 2019 - FGVC6,2201.0


In [35]:
df_teams_per_comp = dt[['title','name_team']].groupby('title').count()
df_teams_per_comp.reset_index(level=0, inplace=True)
df_teams_per_comp.head()

,title,name_team
0,iMaterialist (Fashion) 2019 at FGVC6,242
1,iNaturalist 2019 at FGVC6,216
2,iWildCam 2019 - FGVC6,337


In [36]:
df_total=pd.merge(df_entries_per_comp, df_teams_per_comp, on='title')
df_total

,title,entries,name_team
0,iMaterialist (Fashion) 2019 at FGVC6,3497.0,242
1,iNaturalist 2019 at FGVC6,1477.0,216
2,iWildCam 2019 - FGVC6,2201.0,337


In [37]:
df_total.rename(columns={'name_team':'num_teams'}, inplace=True)                 
                 

In [38]:
df_total

,title,entries,num_teams
0,iMaterialist (Fashion) 2019 at FGVC6,3497.0,242
1,iNaturalist 2019 at FGVC6,1477.0,216
2,iWildCam 2019 - FGVC6,2201.0,337


In [39]:
df_total.to_csv('title_entries_team.csv')

#### Una vez seleccionados los datos y limpio el dataframe 'info_teams_Private', cargamos el 'Dataframe_Competitions_INFO' para ha